<a href="https://colab.research.google.com/github/michliang26/W1D1_PM/blob/main/h8dsft_P0LC2_michelle_liang.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Bab perkenalan
Nama: Michelle Liang
Batch: HCK-009
Objective: Pemahaman konsep-konsep SQL dan Pandas

In [6]:
from google.colab import auth
from google.cloud import bigquery
auth.authenticate_user()
print('Authenticated')

project_id = "live-code-2-400802" #GUNAKAN GCP PROJECT-ID KALIAN MASING-MASING
client = bigquery.Client(project=project_id)

Authenticated


# **Problem 1 - SQL**


1. Apa saja nama perusahaan yang terdapat dalam data?


In [43]:
# answer 1

# Query untuk mencari nama-nama perusahaan yang ada dalam data
df_1 = client.query('''
SELECT company #Tampilkan kolom company
FROM ftds-hacktiv8-project.phase0_ftds_009_hck.taxi_fare
GROUP BY company #Mengelompokkan data yang memiliki nama perusahaan yang sama
''').to_dataframe()
df_1

#Total ada 5 perusahaan yang terdapat dalam data. Nama-nama perusahaannya adalah Deluc Porter, Syntaxi Inc., Atomic Inc., Federway Int. dan Rocket Cab Auto.


,company
0,Deluc Porter
1,Synctaxi Inc.
2,Atomic Inc.
3,Federway Int.
4,Rocket Cab Auto


2.Berapa nilai minimum dan maksimum dari waktu pengambilan data?


In [42]:
# answer 2
df_1 = client.query('''

SELECT
MIN(trip_start_timestamp) as minimum, #Mendapatkan data yang memiliki value paling rendah
MAX(trip_start_timestamp) as maximum  #Mendapatkan data yang memiliki value paling tinggi
FROM ftds-hacktiv8-project.phase0_ftds_009_hck.taxi_fare

''').to_dataframe()
df_1

# Nilai minimum dari pengambilan data yang diambil dari data waktu saat perjalanan dimulai adalah pada tanggal 2 January 2021, pada pukul 11:00.
# Nilai maksimum dari pengambilan data yang diambil dari waktu saat perjalanan dimulai adalah pada tanggal 31 December 2022, pada pukul 14:15.


,minimum,maximum
0,2021-01-02 11:00:00+00:00,2022-12-31 14:15:00+00:00


3.Bandingkan jumlah banyaknya perjalanan antara seluruh perusahaan pada tahun 2022?


In [36]:
# answer 3
df_1 = client.query('''
SELECT company, #Tampilkan kolom company
COUNT(unique_key) as trip_count #Menghitung jumlah data yang ada yang merepresentasikan jumlah banyaknya perjalanan sebuah perusahaan
FROM `ftds-hacktiv8-project.phase0_ftds_009_hck.taxi_fare`
WHERE EXTRACT (year from trip_start_timestamp) = 2022 #Menyaring data yang didapatkan pada tahun 2022
''').to_dataframe()
df_1

# Pada tahun 2022, perusahaan yang memiliki jumlah perjalanan paling banyak adalah Deluc Porter dengan jumlah perjalanan sebanyak 1050.
# Pada tahun 2022, perusahaan yang memiliki jumlah perjalanan kedua paling banyak adalah Synctaxi Inc dengan jumlah perjalanan sebanyak 1341.
# Pada tahun 2022, perusahaan yang memiliki jumlah perjalanan ketiga paling banyak adalah Federway Int.dengan jumlah perjalanan sebanyak 1059.
# Pada tahun 2022, perusahaan yang memiliki jumlah perjalanan ketiga paling sedikit adalah Rocket Cab Auto dengan jumlah perjalanan sebanyak 77.


,company,trip_count
0,Deluc Porter,1050
1,Synctaxi Inc.,1341
2,Federway Int.,1059
3,Rocket Cab Auto,77


4. Perusahaan mana yang memiliki tarif perjalanan per km termurah?

In [37]:
# answer 4

df_1 = client.query('''

SELECT company, fare/trip_km AS fare_per_km
FROM `ftds-hacktiv8-project.phase0_ftds_009_hck.taxi_fare`
ORDER BY fare_per_km ASC
limit 1

''').to_dataframe()
df_1

#Perusahaan yang memiliki tarif perjalanan per km termurah adalah Atomic Inc.	dengan jumlah tarif per km yang mendekati nol.

,company,fare_per_km
0,Atomic Inc.,0.0


# **Problem 2 - Pandas**

In [40]:
# answer 1

# Membuat kolom ketentuan_waktu sesuai dengan ketentuan jam

import pandas as pd

#Load table taxi_fare
df = client.query("""
SELECT *
FROM `ftds-hacktiv8-project.phase0_ftds_009_hck.taxi_fare`
""").to_dataframe()

#Mengekstrak kolom trip_start_timestamp dan mengformat value di kolom menjadi datetime
df['trip_start_timestamp'] = pd.to_datetime(df['trip_start_timestamp'])

#Buat list kosong untuk menampung data jam
keterangan_waktu = []

#Untuk setiap data jam pada tanggal perjalanan
for jam in df['trip_start_timestamp'].dt.hour:
   #Apabila jam 04:00 - 09:59
    if 4 <= jam <= 9.59:
        # Masukkan 'Pagi' kedalam list jam
        keterangan_waktu.append('Pagi')
    #Apabila jam 10:00 - 15:59
    elif 10 <= jam <= 15.59:
        # Masukkan 'Siang' kedalam list jam
        keterangan_waktu.append('Siang')
    #Apabila jam 16:00 - 18:59
    elif 16 <= jam <= 18.59:
        # Masukkan 'Sore' kedalam list jam
        keterangan_waktu.append('Sore')
    #Apabila jam 19:00 - 23:59
    elif 19 <= jam <= 23.59:
        # Masukkan 'Malam' kedalam list jam
        keterangan_waktu.append('Malam')
    #Apabila jam 00:00 - 03:59
    elif 0 <= jam <= 3.59:
        # Masukkan 'Dini hari' kedalam list jam
        keterangan_waktu.append('Dini hari')

#Buat kolom keterangan_waktu yang berisi values dari list keterangan_waktu
df['keterangan_waktu'] = keterangan_waktu
print (df['keterangan_waktu'])


0       Malam
1       Siang
2       Malam
3       Siang
4       Siang
        ...  
9281     Sore
9282    Siang
9283    Siang
9284     Sore
9285    Siang
Name: keterangan_waktu, Length: 9286, dtype: object


2. Berapa total jarak perjalanan yang dicapai perusahaan Federway Int. pada tahun 2022? (Gunakan pandas query untuk menjawab soal no. 2)


In [41]:
# answer 2

import pandas as pd

# Load table taxi_fare
df = client.query("""
SELECT *
FROM `ftds-hacktiv8-project.phase0_ftds_009_hck.taxi_fare`
""").to_dataframe()

# Filter untuk mencari data perusahaan Federway Int. pada tahun 2022
filtered_data = df[(df['company'] == 'Federway Int.') &
                    (df['trip_end_timestamp'].dt.year == 2022)]

# Hitung total jarak perjalanan
total_jarak_perjalanan= filtered_data['trip_km'].sum()

# Tampilkan hasil
print(total_jarak_perjalanan)

# Total jarak perjalanan yang dicapai perusahaan Federway Int. pada tahun 2022 adalah 4359.24135

4359.24135
